In [24]:
from ultralytics import YOLO
import os
import numpy as np

In [25]:
# Load the YOLO-Pose model
model = YOLO("yolov8n-pose.pt")  # Use your trained YOLO-Pose model

In [26]:
# Define dataset folders
image_folders = {
    # commented out for now, as runs folder kept messing up after one data set is over, had to rename runs folder each time and then run the code again one dataset at a time
    "train": "filtered_images/train",
    # "val": "filtered_images/val",
    # "test": "filtered_images/test"
}

output_base = "runs/pose/predict"

In [27]:
# Function to clean YOLO-Pose annotations
def clean_yolo_pose_annotations(labels_folder, expected_keypoints=17, min_keypoints=5):
    if not os.path.exists(labels_folder):
        print(f"Skipping {labels_folder}: No annotation files found.")
        return

    for label_file in os.listdir(labels_folder):
        label_path = os.path.join(labels_folder, label_file)

        with open(label_path, "r") as f:
            lines = f.readlines()

        cleaned_lines = []

        for line in lines:
            data = list(map(float, line.strip().split()))

            # Dynamically detect the number of keypoints
            num_keypoints = (len(data) - 1) // 3  # Ignore class ID

            if num_keypoints < expected_keypoints:
                print(f"Skipping {label_file}: Detected only {num_keypoints} keypoints instead of {expected_keypoints}.")
                continue  # Skip if keypoints are missing
            
            keypoints = np.array(data[1:1 + num_keypoints * 3]).reshape(-1, 3)  # Reshape dynamically
            
            # Remove keypoints with confidence == 0
            valid_keypoints = keypoints[keypoints[:, 2] > 0]

            # Keep only annotations with at least `min_keypoints`
            if len(valid_keypoints) >= min_keypoints:
                cleaned_lines.append("0 " + " ".join(map(str, valid_keypoints.flatten())) + "\n")

        # If cleaned annotations exist, overwrite the file
        if cleaned_lines:
            with open(label_path, "w") as f:
                f.writelines(cleaned_lines)
        else:
            print(f"Deleting {label_file}: No valid keypoints found.")
            os.remove(label_path)  # Delete file if no valid keypoints remain

# Run YOLO-Pose inference on all datasets
for dataset, img_folder in image_folders.items():
    output_path = os.path.join(output_base, dataset)
    labels_path = os.path.join(output_path, "labels")

    print(f"Running YOLO-Pose on {dataset} dataset...")
    
    # Run YOLO-Pose inference
    results = model.predict(source=img_folder, save=True, save_txt=True, save_conf=True, project=output_base, name=dataset)

    # Clean YOLO-Pose annotations
    clean_yolo_pose_annotations(labels_path)

    print(f"✅ {dataset} dataset processed and cleaned!")

print("🎉 YOLO-Pose inference and cleaning completed for all datasets!")

Running YOLO-Pose on train dataset...


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/10360 c:\Users\ashan\Documents\IIT\5th yr\My FYP\Feb 2025\YOLO-BarbellRow\filtered_images\train\52723_8_10.jpg: 640x448 4 persons, 114.7ms
image 2/10360 c:\Users\ashan\Documents\IIT\5th yr\My FYP\Feb 2025\YOLO-BarbellRow\filtered_images\train\52723_8_12.jpg: 640x448 3 persons, 111.9ms
image 3/10360 c:\Users\ashan\Documents\IIT\5th yr\My FYP\Feb 2025\YOLO-BarbellRow\filtered_images\train\52723_8_19.jpg: 640x44

In [28]:
# # Run YOLO-Pose inference on all datasets
# for dataset, img_folder in image_folders.items():
#     output_path = os.path.join(output_base, dataset)
#     labels_path = os.path.join(output_path, "labels")

#     print(f"Running YOLO-Pose on {dataset} dataset...")
    
#     # Run YOLO-Pose inference
#     results = model.predict(source=img_folder, save=True, save_txt=True, save_conf=True, project=output_base, name=dataset)

#     # Clean YOLO-Pose annotations
#     def clean_yolo_pose_annotations(labels_folder, min_keypoints=5):
#         if not os.path.exists(labels_folder):
#             print(f"Skipping {labels_folder}: No annotation files found.")
#             return

#         for label_file in os.listdir(labels_folder):
#             label_path = os.path.join(labels_folder, label_file)

#             with open(label_path, "r") as f:
#                 lines = f.readlines()

#             cleaned_lines = []
            
#             for line in lines:
#                 data = list(map(float, line.strip().split()))

#                 if len(data) < 52:  # Ensure correct number of keypoints (17 * 3 + 1)
#                     print(f"Skipping {label_file}: Incorrect number of keypoints.")
#                     continue
                
#                 keypoints = np.array(data[1:]).reshape(-1, 3)  # Reshape to (num_keypoints, 3)
                
#                 # Remove keypoints with confidence == 0
#                 valid_keypoints = keypoints[keypoints[:, 2] > 0]
                
#                 # Keep only annotations with at least `min_keypoints`
#                 if len(valid_keypoints) >= min_keypoints:
#                     cleaned_lines.append(line)

#             # If cleaned annotations exist, overwrite the file
#             if cleaned_lines:
#                 with open(label_path, "w") as f:
#                     f.writelines(cleaned_lines)
#             else:
#                 print(f"Deleting {label_file}: No valid keypoints found.")
#                 os.remove(label_path)  # Delete file if no valid keypoints remain

#     # Run annotation filtering
#     clean_yolo_pose_annotations(labels_path)

#     print(f"✅ {dataset} dataset processed and cleaned!")

In [29]:
print("🎉 YOLO-Pose inference and cleaning completed for all datasets!")

🎉 YOLO-Pose inference and cleaning completed for all datasets!
